## Load Libraries

In [1]:
import numpy as np
import pandas as pd
import json

## Read and Transform the Excel Files

In [2]:
# google spread sheet data

files = ['Fall 2015.xlsx', 'Spring 2017.xlsx']#, 'Spring 2016.xlsx']
google_sheets_transformed = pd.DataFrame(columns=['Class', 'Description', 'Start date', 'Start time', 'End date', 'End time', 'Duration'])

for file in files:
    sheets = pd.ExcelFile(file).sheet_names
    
    for sheet in sheets:
        # load in the data
        original = pd.read_excel(file, sheet_name=sheet)

        # get the class names
        classNames = original.keys()[1:]

        # create a new dataframe
        new = pd.DataFrame(columns=['Class', 'Description', 'Start date', 'Start time', 'End date', 'End time', 'Duration'])

        # transform the data into the correct format
        new['Class'] = np.ravel([[className]*len(original) for className in classNames])
        new['Start date'] = pd.Series(np.concatenate([original['Date'].values]*len(classNames)))
        new['End date'] = pd.Series(np.concatenate([original['Date'].values]*len(classNames)))
        new['Duration'] = pd.to_timedelta(np.ravel([original[className].values for className in classNames]), unit='h')
        # remove data with 0 for the Duration, and resort and index
        new = new[new['Duration'] > pd.Timedelta(0)]

        currentLen = len(google_sheets_transformed)
        new.index = range(currentLen, currentLen + len(new))

        # append to the main DataFrame
        google_sheets_transformed = google_sheets_transformed.append(new)

In [3]:
# Toggle data
toggle_data = pd.read_csv('Toggl_time_entries_2017-01-01_to_2019-01-01.csv')
# conver the duration
toggle_data['Duration'] = pd.to_timedelta(toggle_data['Duration'])

## Add to Toggle Data

In [4]:
# combine the two dataframes
data = pd.concat([
                  google_sheets_transformed,
                  ]
                ).sort_values(by=['Start date', 'Class'])

# conver to seconds
data['Duration'] = pd.to_numeric(data['Duration'])/1000000000

In [5]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        elif isinstance(obj, (pd.Timestamp)):
            return obj.isoformat()
            
        return json.JSONEncoder.default(self, obj)

In [6]:
dumped = json.dumps(data.values, cls=NumpyEncoder)

with open('data.json', 'w') as f:
    json.dump(json.loads(dumped), f)